<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danh sách nhóm
- Đoàn Xuân Minh - MSSV: 18521093
- Lê Trần Phúc Nguyên - MSSV: 18521157
- Phan Quang Tấn - MSSV: 18521377


# Mô tả bài toán : Dự đoán chủ đề của một bài báo tiếng Anh
- Input: Một đoạn văn bản của một bài báo tiếng Anh (article)
- Output: Chủ đề của đoạn văn bản 

Code tham khảo: https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db

Code tham khảo: https://pythonspot.com/nltk-stop-words/

# Prepare dataset

Thực hiện cào dữ liệu (crawl data) từ các trang báo tiếng Anh nổi tiếng như : Dailymail https://www.dailymail.co.uk/home/index.html, Telepraph http://telegraph.co.uk/

Code Crawl : https://github.com/tankien76/CS114.K21/blob/master/CrawlerTest.ipynb

Tập dataset sau khi crawl gồm có hơn 2600 bài báo với 8 chủ đề khác nhau như : sport, tvshowbiz, femail,.... Mỗi chủ đề gồm có 350 bài báo.

Xác thực và tạo ứng dụng PyDrive.

In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1o2zIVZBmqSJa8huDq_Nn8waeTksbH4KI"})
downloaded.GetContentFile('text_data.csv')

Import các thư viện:
- pandas : đọc dữ liệu file json
- numpy : xử lý dữ liệu text

In [ ]:
import pandas as pd, numpy as np, re

Đọc dữ liệu bằng thư viện pandas từ file text_data.csv

In [ ]:
dataset = pd.read_csv("text_data.csv", encoding='ISO 8859-1')
dataset.head()

,article,category
0,PHOTOS of McDonalds restaurants and merchandis...,lifestyle
1,"ARIESMarch 21 to April 20SATURN, the most powe...",lifestyle
2,BRITS are being tempted to fly abroad once loc...,lifestyle
3,DONNA Mitchell tried every fad diet in the boo...,lifestyle
4,FASHION fans can buy jeans and a sleeveless to...,lifestyle


Đưa dữ liệu article và category vào list tương ứng

In [ ]:
features = dataset['article']
labels = dataset['category']

Thực hiện chuyển đổi các nhãn dán thành số để dễ dàng thực hiện trong khi training và test model

In [ ]:
from sklearn import preprocessing
number = preprocessing.LabelEncoder()
labels= number.fit_transform(labels)

# Feature Engineering
Tiến hành xử lý dữ liệu văn bản (article) :

- Loại bỏ các ký tự đặc biệt(dấu câu, số)
- Loại bỏ các stopword trong tiếng anh
- Vector hóa dữ liệu

download punkt trong thư viện nltk để tách từ khi xử lý dữ liệu

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

download stopwords trong thư viện nltk để xử lý các từ không cần thiết trong tiếng anh

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Import thư viện nltk để sử lý ngôn ngữ tự nhiên

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Loại bỏ các ký tự đặc biệt như: dấu câu, số,...

In [ ]:
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Loại bỏ các stopword (các từ không ảnh hưởng đến nội dung của văn bản) trong tiếng Anh

In [ ]:
stop_words = set(stopwords.words('english'))
for i in range (len(features)):
  word_tokens = word_tokenize(features[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu văn bản bằng TF-IDF:
- TF: tần suất xuất hiện của 1 từ trong 1 văn bản
- IDF: tần suất nghịch của 1 từ trong 1 tập văn bản
- .fit: Máy học từ vựng và tính toán idf( tần suất nghịch của 1 từ trong tập các văn bản) của train set.

Tham khảo: https://towardsdatascience.com/how-sklearns-tf-idf-is-different-from-the-standard-tf-idf-275fa582e73d

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer()
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Đưa dữ liệu văn bản về matrix để training model 

In [ ]:
tf_idf_x = tfIdfVec.transform(features)

Tiến hành chia dữ liệu train và test: theo tỉ lệ 80% training và 20% testing vì khi thực hiện theo tỉ lệ 70/30 thì độ chính xác của các thuật toán giảm

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.toarray(), labels, test_size = .2, random_state = 0)

# Chosing Model

Thêm thư viện sklearn gồm :
- Các model sử dụng : LinearSVC, Multinomial Naive Bayes, Random Forest

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Model Linear support classifier

In [ ]:
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print("Train accuracy: ",lsvc.score(features_train, labels_train))
print("Test accuracy: ",lsvc.score(features_test, labels_test))

Train accuracy:  0.9985775248933144
Test accuracy:  0.8920454545454546


Model Multinomial Naive Bayes

In [ ]:
mnb = MultinomialNB()
mnb.fit(features_train, labels_train)
print("train accuracy: ",mnb.score(features_train, labels_train))
print("Test accuracy: ",mnb.score(features_test, labels_test))

train accuracy:  0.9193930772878142
Test accuracy:  0.8352272727272727


Model Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print("train accuracy: ",rfc.score(features_train, labels_train))
print("Test accuracy: ",rfc.score(features_test, labels_test))

train accuracy:  0.9957325746799431
Test accuracy:  0.7234848484848485


# Đánh giá Model

Để đánh giá performance của các model thì cách tính bằng accuracy không phản ánh chính xác dữ liệu được phân loại như thê nào vì vậy chúng ta cần 1 cách tính độ chính xác khác đó là **F1 Score**

F1-score: trung bình điều hòa giữa precision và recall

F1-score = 2*(precision * recall)/(precision+recall)

**Precision** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm được phân loại là positive

Precision = TP/(TP+FP)
Với : TP(True positive), FP(False positive)

**Recall** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm thực sự là positive

Recall=TP/(TP+FN) Với TP(True positive), FN(False negative)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

Model Linear support classifier

In [ ]:
y1_pred=lsvc.predict(features_test)
f1 = f1_score(labels_test, y1_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.8861288110545362


Model Multinomial Naive Bayes

In [ ]:
y_pred=mnb.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.7461226600644999


Model Random Forest Classifier

In [ ]:
y_pred=rfc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.7052716389326805


In [ ]:
confusion = confusion_matrix(labels_test, y1_pred)
print('Confusion Matrix')
print(confusion)

Confusion Matrix

[[52  1 10  0  0  0  1  5]
 [ 3 70  0  0  0  0  2  2]
 [ 5  1 52  1  2  2  2  1]
 [ 1  0  2 17  0  0  2  0]
 [ 0  0  1  0 60  0  0  0]
 [ 0  0  1  0  0 84  1  0]
 [ 4  3  2  0  0  2 72  0]
 [ 0  0  0  0  0  0  0 64]]


In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(labels_test, y1_pred, target_names=['femail', 'films', 'health','lifestyle','sport','technology','travel','tvshowbiz']))


Classification Report

              precision    recall  f1-score   support

      femail       0.80      0.75      0.78        69
       films       0.93      0.91      0.92        77
      health       0.76      0.79      0.78        66
   lifestyle       0.94      0.77      0.85        22
       sport       0.97      0.98      0.98        61
  technology       0.95      0.98      0.97        86
      travel       0.90      0.87      0.88        83
   tvshowbiz       0.89      1.00      0.94        64

    accuracy                           0.89       528
   macro avg       0.89      0.88      0.89       528
weighted avg       0.89      0.89      0.89       528

